### NOTEBOOK for constructiong POD-RBF NIROM approximation for a shallow water example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. RBF interpolation is adopted to approximate the evolution of the time dynamics in the reduced space spanned by the POD modes.  



In [ ]:
## Load modules
import numpy as np
import scipy
from importlib import reload

import os
import gc
from importlib import reload
from scipy import interpolate
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, ScalarFormatter, FormatStrFormatter

from matplotlib import animation
matplotlib.rc('animation', html='html5')
from IPython.display import display
import matplotlib.ticker as ticker
from matplotlib import rcParams
from matplotlib.offsetbox import AnchoredText

# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, # fontsize for x and y labels (was 10)
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

import itertools
colors = itertools.cycle(['r','g','b','m','y','c'])
markers = itertools.cycle(['p','d','o','^','s','x','D','H','v','*'])

base_dir = os.getcwd()
work_dir = os.path.join(base_dir,'../examples/')
data_dir = os.path.join(base_dir,'../data/')
nirom_data_dir = os.path.join(base_dir,'../data/')
fig_dir = os.path.join(base_dir,'../figures/podrbf/')

import pynirom
from pynirom.pod import pod_utils as pod
from pynirom.rbf import main as prb
from pynirom.rbf import rbf as rbf
from pynirom.rbf import plotting as pu
from pynirom.utils import data_utils as du



In [ ]:
## Load snapshot data
## Please download the data files from 
## https://drive.google.com/drive/folders/1yhudg8RPvwV9SJx9CTqANEnyN55Grzem?usp=sharing
## and place them in <data_dir>

# ### San Diego problem
# model = 'SD'
# data = np.load(os.path.join(data_dir,'san_diego_tide_snapshots_T4.32e5_nn6311_dt25.npz'))
# mesh = np.load(os.path.join(data_dir,'san_diego_mesh.npz'))

### Red River problem
model ='Red'
data = np.load(os.path.join(data_dir,'red_river_inset_snapshots_T7.0e4_nn12291_dt10.npz'))
mesh = np.load(os.path.join(data_dir,'red_river_mesh.npz'))

print("Solution component keys are : " + str(list(data.keys())))
print("Mesh element keys are : " + str(list(mesh.keys())))

print('\nHFM data has {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    data['T'].shape[0],data['S_dep'].shape[0],
                    data['T'][0], data['T'][-1]))


In [ ]:
## Prepare training snapshots
print('\n-------Prepare training and testing data---------')
soln_names = ['S_dep', 'S_vx', 'S_vy']
comp_names={0:'S_dep',1:'S_vx',2:'S_vy'}
Nc=3 
nodes = mesh['nodes']; triangles = mesh['triangles']
Nn = nodes.shape[0]; Ne = triangles.shape[0]

snap_start = 100
if model == 'SD':
    T_end = 50*3600   ### 50 hours in seconds
    snap_incr=4
elif model == 'Red':
    T_end = 3.24e4    ### 9 hours in seconds
    snap_incr=3

## --- Prepare training snapshots
snap_train, times_train = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_train.shape[0],
                                        times_train[0]/3600, times_train[-1]/3600))
Nt_b = times_train.size


## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr - 2
snap_pred_true, times_online = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, 
                                                    incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_online.shape[0],
                                        times_online[0]/3600, times_online[-1]/3600))
Nt_online=times_online.size

## ------- Save full HFM data without spinup time -----
snap_data, times_offline = du.prepare_data(data, soln_names, start_skip=snap_start,)




del mesh
gc.collect()

In [ ]:
## Instantiate the POD-RBF class

if model == 'SD':
    trunc_lvl = 0.9999995
    eps = 0.01
    
elif model == 'Red':
    trunc_lvl = 0.99
    eps = 0.05
    
PRB = prb.PODRBFBase(trunc = trunc_lvl)

In [ ]:
## Compute the POD basis for the space of snapshots
Phi, Sigma, Z_train = PRB.compute_pod_basis(snap_train, times_train)
dzdt_train = PRB._dzdt_train

## Compute the RBF approximation of the time derivative of
## POD modal coefficients

if model == 'Red':
    rbf_incr = snap_incr+2
elif model == 'SD':
    rbf_incr = snap_incr-1

S_rbf, times_rbf = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, 
                                                    incr=rbf_incr)

Z_rbf = pod.project_onto_basis(S_rbf,PRB.basis, PRB._S_mean)
A_rbf, rbf_centers_rbf, rbf_coeff_rbf = PRB.fit_rbf(Z_rbf, times_rbf, kernel='matern',eps=eps)

del data
gc.collect()

In [ ]:
## Evaluate NIROM online at queried time points

%time uh, zh = PRB.predict_time(times_online)


In [ ]:
def set_label(key):
    if key == 'S_vx':
        ky = 'u'
    elif key == 'S_vy':
        ky = 'v'
    elif key == 'S_dep':
        ky = 'h'
    return ky

### Visualize domain and ROM results

In [ ]:
## --- Visualize the singular values

pu.plot_sing_val(Sigma)

# os.chdir(fig_dir)
# plt.savefig('%s_sing_value.png'%model,dpi=300,bbox_inches='tight')

In [ ]:
## --- Visualize l2-norm of time derivative of modal coefficients

dZdata_norm = {}
for ii,key in enumerate(soln_names):
    dZdata_norm[key] = np.linalg.norm(dzdt_train[key],axis=0);

fig = plt.figure(figsize=(12,3))
ky = soln_names[1]; ky1 = soln_names[2]; ky2 = soln_names[0]
t_indx = times_train[:-1]/3600
plt.plot(t_indx[:],(dZdata_norm[ky][:]),'r-o', markersize=8,markevery=100,label='$u_x$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky1][:]),'g-D', markersize=8,markevery=100,label='$u_y$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky2][:]),'b-^', markersize=8,markevery=100,label='$p$',linewidth=2)

ax = plt.gca()
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
lg = plt.legend(fontsize=16,ncol=3,loc='upper right') 
plt.xlabel('Time (hours)',fontsize=16)


In [ ]:
## Compute spatial RMS/Relative error
    
rms_err = PRB.compute_error(snap_pred_true, uh, soln_names, metric='rms')

In [ ]:
## --- Compare the HFM and NIROM solution

key = 'S_vx'; iplot = 1200
iplot_true = np.argmin(np.abs(times_offline-times_online[iplot]))

fig = plt.figure(figsize=(16,6))
urom = uh[key][:,iplot]
usnap = snap_data[key][:,iplot_true]
print("comparing NIROM solution at t = {1:.2f} hrs and fine-grid solution at t = {0:.2f} hrs".format(
                                            times_offline[iplot_true]/3600, times_online[iplot]/3600))

ax1 = plt.subplot(1,2,1); ax1.axis('off')
surf1, boundaries_interp = pu.viz_sol(urom,nodes,triangles)
ax1.set_title("NIROM solution at t = %.2f hrs\n $%1.5f<\mathbf{%s}<%1.5f$"%(times_online[iplot]/3600,
                                                            np.amin(urom),set_label(key),
                                                             np.amax(urom)),fontsize=16)
plt.axis('off')
plt.colorbar(surf1, shrink=0.8,aspect=20, pad = 0.03)


ax2 = plt.subplot(1,2,2); ax2.axis('off')
surf2, boundaries_true = pu.viz_sol(usnap,nodes,triangles)
ax2.set_title("HFM solution at t = %.2f hrs\n $%1.5f<\mathbf{%s}<%1.5f$"%(times_online[iplot]/3600, 
                                                            np.amin(usnap),set_label(key),
                                                             np.amax(usnap)),fontsize=16)
plt.axis('off')
plt.colorbar(surf2, shrink=0.8,aspect=20, pad = 0.03)



In [ ]:
### Visualize spatial RMS/Relative errors for all solution components

x_inx = times_online/3600
tr_mark = np.searchsorted(times_online, times_train[-1])

vstring = {}
for key in soln_names:
    vstring[key] = set_label(key)
    
pu.plot_RBF_err(rms_err, x_inx, soln_names, vstring, unit='hours', metric = 'rms' )

# os.chdir(fig_dir)
# plt.savefig('%s_rbf_rms_tskip%d_oskip%d.pdf'%(model,snap_incr,pred_incr), bbox_inches='tight')

In [ ]:
## Save the NIROM solutions to disk
save_nirom_solutions = False

if save_nirom_solutions:
    os.chdir(nirom_data_dir)
    np.savez_compressed('%s_online_rbf'%model,S_dep=uh['S_dep'],S_vx = uh['S_vx'], S_vy = uh['S_vy'],
                                            zS_dep=zh['S_dep'],zS_vx = zh['S_vx'], zS_vy = zh['S_vy'],
                                            time=times_online)

    os.chdir(work_dir)

In [ ]:
## Saving the ROM model
# os.chdir(data_dir)
# filename='podrbf_rom_sandiego'
# filename='podrbf_rom_redriver'
# PRB.save_to_disk(filename,PRB)